In [ ]:
! uv pip install langchain langchain-community markitdown[all]  langchain-text-splitters bs4 requests

In [ ]:
! uv pip install -q -U google-genai

In [ ]:
! uv pip list

In [ ]:
import re

def remove_markdown_h1(text: str) -> str:
    if not text:
        return ""

    # Xóa các dòng bắt đầu bằng đúng 1 dấu #
    return re.sub(
        r"(?m)^\s{0,3}#\s+.*$",
        "",
        text
    ).strip()

In [ ]:
import os
from markitdown import MarkItDown

def convert_to_markdown(input_filepath):
    md = MarkItDown(enable_plugins=False)
    result = md.convert(input_filepath)
    return result.text_content
    

In [ ]:
from google import genai
from dotenv import load_dotenv
import re
load_dotenv()
def convert_gemini(text, retry = 3):
    
    prompt = f"""
    Bạn là một trợ lý chuyên cấu trúc hóa văn bản sang Markdown theo nguyên tắc ngữ nghĩa chặt chẽ.

    NHIỆM VỤ:
    Chuyển đổi văn bản đầu vào thành Markdown rõ ràng, mạch lạc, hợp lý về mặt ngữ nghĩa.
    Tuyệt đối chỉ chỉnh sửa nội dung khi sai chính tả hay ngữ pháp hay có lỗi xuống dòng.
    KHÔNG lạm dụng header.
    KHÔNG tạo header rỗng hoặc header chỉ để gom nhóm hình thức.

    NGUYÊN TẮC CỐT LÕI (BẮT BUỘC):
    ❗ Mỗi header phải đại diện cho MỘT khối nội dung hoàn chỉnh.
    ❗ Header phải có nội dung trực tiếp đi kèm ngay bên dưới.
    ❗ Không được tạo header chỉ để làm nhãn cho các header con.

    QUY TẮC CẤP HEADER CƠ BẢN (CHỈ ÁP DỤNG KHI ĐỦ ĐIỀU KIỆN NGỮ NGHĨA):
    - I, II, III, … → ##
    - 1, 2, 3, … → ###
    - a, b, c, … → ####

    QUY TẮC THEO ĐỘ DÀI & LOẠI THÔNG TIN:
    KHÔNG tạo header cho các dòng chỉ là:
    - Mốc thời gian (ngày / tháng / năm)
    - Sự kiện ngắn gắn với mốc thời gian
    - Dòng dạng “Ngày – Sự kiện”
    - Các ý song song trong danh sách mục tiêu, nhiệm vụ, tiêu chí

    Các dòng này phải được xem là:
    → nội dung thuộc cùng một mục
    → trình bày dưới dạng đoạn văn hoặc danh sách (numbered list / bullet)

    VÍ DỤ KHÔNG ĐƯỢC COI LÀ HEADER:
    - 07/09/1953 – Thành lập ...
    - Năm 1997: Công bố quyết định ...
    - 22/03/1999 Thành lập trung tâm ...
    - 1. Phát triển …
    - 2. Nâng cao …

    QUY TẮC RIÊNG CHO TIMELINE / LỊCH SỬ:
    - Nếu một header (## hoặc ###) bao gồm nhiều mốc thời gian liên tiếp:
    - Giữ DUY NHẤT header đó
    - Toàn bộ mốc thời gian bên trong KHÔNG được đánh header con
    - Trình bày các mốc dưới dạng danh sách (-) hoặc đoạn văn

    QUY TẮC TẠO HEADER CON:
    CHỈ tạo header con khi ĐỒNG THỜI thỏa mãn:
    - Header cha có nội dung mô tả riêng, hoàn chỉnh
    - Dòng được chọn làm header mang tính khái quát (chủ đề, giai đoạn, phân loại)
    - Nội dung bên dưới đủ dài (≥ 3 câu hoặc 3 ý)
    - Có khả năng chia tiếp thành các ý độc lập

    Nếu KHÔNG chắc chắn:
    → Ưu tiên KHÔNG tạo header
    → Ưu tiên trình bày dưới dạng nội dung thường

    QUY TẮC SUY LUẬN KHI VĂN BẢN KHÔNG RÕ CẤU TRÚC:
    - Dòng mang tính bao trùm, khái quát → header
    - Dòng mang tính dữ kiện, mô tả, minh họa → nội dung
    - Một header chỉ chứa duy nhất 1 dòng sự kiện → hạ cấp thành nội dung
    
    QUY TẮC BỔ SUNG CHO VĂN BẢN TIN TỨC:

    - Mặc định coi toàn bộ bài tin là MỘT document liền mạch.
    KHÔNG bắt buộc tạo header.

    - CHỈ tạo header (##) khi có một khối ngữ cảnh hoặc nội dung tổng quát
    độc lập về mặt ngữ nghĩa, đứng riêng được, và có tối thiểu 3 câu hoặc 3 ý.

    -KHÔNG tạo header cho:
    + mốc thời gian, lễ ký kết, diễn biến sự kiện
    + phát biểu cá nhân, trích dẫn
    + chú thích ảnh, danh sách người tham dự
    + đoạn PR, nhận định chung, kỳ vọng

    -Nếu không chắc chắn việc tạo header có cải thiện cấu trúc ngữ nghĩa
    và khả năng chunk dữ liệu → KHÔNG tạo header, giữ nguyên dạng đoạn văn.


    KHÔNG ĐƯỢC:
    - Để header không có nội dung trực tiếp
    - Tạo cấu trúc kiểu:
    ## Tiêu đề
    ### 1.
    ### 2.
    - Biến timeline hoặc danh sách mục tiêu thành cây header sâu
    - Làm mất nội dung gốc
    - Thêm nội dung mới
    
    ĐẦU VÀO:
    {text}

    ĐẦU RA:
    Chỉ trả về nội dung Markdown đã xử lý.
    Không giải thích.
    Không nhận xét.

    """
    client = genai.Client(api_key=os.getenv("GENAI_API_KEY"))
    while retry > 0:
        try:
             response = client.models.generate_content(
                model="gemini-3-flash-preview", contents=prompt
            )
             break
        except Exception as e:
            retry -= 1
            if retry == 0:
                raise e
    markdown_pattern_cell = r'```markdown(.*?)```'
    result = re.sub(markdown_pattern_cell, "", response.text, flags=re.DOTALL)
    result = remove_markdown_h1(result)
    print(result)
    return result

In [ ]:
import re

def cleantable_markdown(markdown_text):
    """Convert markdown tables in `markdown_text` to clean plain-text rows.

    - Preserves fenced code blocks (```...```) unchanged.
    - Detects markdown tables with a header line containing "|" and a following
      separator line of dashes/colons (with or without surrounding pipes).
    - Replaces each table with its rows where cells are joined by 4 spaces.

    Returns the cleaned text.
    """
    if not markdown_text:
        return ''

    lines = markdown_text.splitlines()
    out_lines = []
    in_code = False
    i = 0

    sep_re = re.compile(r'^\s*\|?\s*[:\-]{1,}\s*(\|\s*[:\-]{1,}\s*)+\|?\s*$')

    def parse_row(r):
        cells = [c.strip() for c in r.split('|')]
        # remove empty leading/trailing cell if line used surrounding pipes
        if cells and cells[0] == '':
            cells = cells[1:]
        if cells and cells[-1] == '':
            cells = cells[:-1]
        # join non-empty cells with some spacing
        return '    '.join([c for c in cells if c != ''])

    while i < len(lines):
        line = lines[i]
        # toggle fenced code blocks
        if line.strip().startswith('```'):
            in_code = not in_code
            out_lines.append(line)
            i += 1
            continue

        if in_code:
            out_lines.append(line)
            i += 1
            continue

        # detect markdown table header + separator
        if '|' in line and (i + 1) < len(lines) and sep_re.match(lines[i + 1]):
            # header
            out_lines.append(parse_row(line))
            # consume separator
            j = i + 2
            # following rows while they contain pipe characters
            while j < len(lines) and ('|' in lines[j]):
                out_lines.append(parse_row(lines[j]))
                j += 1
            i = j
            continue

        # otherwise normal line
        out_lines.append(line)
        i += 1

    return '\n'.join(out_lines)


In [ ]:
import re
import unicodedata

def clean_markdown(md: str) -> str:
    """Clean markdown to plain text for pure textual data (no code, no math)."""
    if not md:
        return ""

    text = unicodedata.normalize("NFKC", md)

    # 1. Remove HTML tags
    text = re.sub(r"<[^>]+>", " ", text)

    # 2. Remove images: ![alt](url) -> alt
    text = re.sub(r"!\[([^\]]*)\]\([^)]+\)", r"\1", text)

    # 3. Remove links: [text](url) -> text
    text = re.sub(r"\[([^\]]+)\]\([^)]+\)", r"\1", text)

    # 5. Remove blockquotes
    text = re.sub(r"^\s*>\s*", "", text, flags=re.MULTILINE)

    # 6. Remove list markers (-, *, +)
    text = re.sub(r"^\s*[-*+]\s+", "", text, flags=re.MULTILINE)

    # 7. Remove emphasis markers (**bold**, *italic*, __bold__, _italic_)
    text = re.sub(r"\*\*(.*?)\*\*", r"\1", text)
    text = re.sub(r"\*(.*?)\*", r"\1", text)
    text = re.sub(r"__(.*?)__", r"\1", text)
    text = re.sub(r"_(.*?)_", r"\1", text)

    # 8. Remove inline code backticks
    text = re.sub(r"`([^`]*)`", r"\1", text)

    # 9. Remove horizontal rules
    text = re.sub(r"^\s*[-*_]{3,}\s*$", "", text, flags=re.MULTILINE)

    # 10. Replace common HTML entities
    text = re.sub(r"&nbsp;|&amp;|&lt;|&gt;", " ", text)

    # 11. Normalize whitespace
    text = re.sub(r"[ \t]+", " ", text)
    text = re.sub(r"\n{3,}", "\n\n", text)

    # 12. Trim lines
    text = "\n".join(line.strip() for line in text.splitlines())

    return text.strip()


In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
def split_by_recursive(text):
    """Split text into chunks using RecursiveCharacterTextSplitter."""
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200,
        separators=["\n\n", "\n", " ", ""],
        length_function=len,
        is_separator_regex=False
    )
    
    chunks = text_splitter.split_text(text)
    return chunks


In [ ]:
from langchain_text_splitters import MarkdownHeaderTextSplitter

def split_by_markdown_header(markdown_text):
    headers_to_split = [
        ("#","Header 1"),
        ("##", "Header 2"),
        ("###", "Header 3"),
        ("####", "Header 4")
    ]
    md_headsplitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split,
                                          strip_headers=True)
    md_header_splits = md_headsplitter.split_text(markdown_text)
    return md_header_splits

In [ ]:
import re
import unicodedata

def convert_md_to_text(md: str) -> str:
    if not md:
        return ""

    text = unicodedata.normalize("NFKC", md)

    text = re.sub(r"<[^>]+>", "", text)
    text = re.sub(r"```[\s\S]*?```", "", text)

    # Remove headings ###
    text = re.sub(r"^#{1,6}\s*", "", text, flags=re.MULTILINE)

    # Remove blockquotes >
    text = re.sub(r"^\s*>+\s?", "", text, flags=re.MULTILINE)

    # Remove list markers
    text = re.sub(r"^\s*[-*+]\s+", "", text, flags=re.MULTILINE)
    text = re.sub(r"^\s*\d+\.\s+", "", text, flags=re.MULTILINE)

    # Remove inline formatting
    text = re.sub(r"\*\*(.*?)\*\*", r"\1", text)
    text = re.sub(r"\*(.*?)\*", r"\1", text)
    text = re.sub(r"`([^`]*)`", r"\1", text)
    text = re.sub(r"~~(.*?)~~", r"\1", text)

    # Links
    text = re.sub(r"\[([^\]]+)\]\([^)]+\)", r"\1", text)

    # Entities
    text = re.sub(r"&nbsp;|&amp;|&lt;|&gt;", " ", text)

    # Whitespace
    text = re.sub(r"[ \t]+", " ", text)
    text = re.sub(r"\n{3,}", "\n\n", text)

    return text.strip()


In [ ]:
def get_name_file(filepath: str) -> str:
    """Extract the file name without extension from a given file path."""
    base_name = os.path.basename(filepath)
    name, _ = os.path.splitext(base_name)
    return name.strip()

In [ ]:
import json
from pathlib import Path
import os

def get_data_path():
    try:
        base = Path(__file__).parent
    except NameError:
        base = Path.cwd()
    data_path = (base / "data/ptit").resolve()
    if not data_path.exists():
        # fallback to workspace-specific data folder (adjust if needed)
        data_path = Path("d:/crapy/crawler/crawler/chunking/data/ptit").resolve()
    return data_path

path = get_data_path()
print("Using data path:", path)

for file_path in path.iterdir():
    print(file_path)
    markdown_text = convert_to_markdown(str(file_path))
    markdown_text = cleantable_markdown(markdown_text)
    markdown_text = convert_gemini(markdown_text)
    clean_md = clean_markdown(markdown_text)
    markdown_chunks = split_by_markdown_header(clean_md)
    file_name = get_name_file(str(file_path))
    for chunk in markdown_chunks:
        content = convert_md_to_text(chunk.page_content)
        # Sửa: dùng .values() để lấy nội dung header thay vì keys
        headers = [convert_md_to_text(v).strip() for v in chunk.metadata.values()] if chunk.metadata else []
        data = {
            "source": file_name,
            "content": content,
            "header": headers
        }
        with open("data_chunked.json", "a", encoding="utf-8") as f:
            f.write(json.dumps(data, ensure_ascii=False) + "\n")

